In [1]:
import open3d as o3d
import numpy as np

def vis_grasp(grasp):
    vis_sphere = []
    for point in grasp:
        sp = o3d.geometry.TriangleMesh.create_sphere(0.01)
        sp.translate(point[:3])
        vis_sphere.append(sp)
    return vis_sphere

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
cropped_pcd = o3d.io.read_point_cloud("../seeds/pointclouds/pose_5_pcd.ply")
o3d.visualization.draw_geometries([cropped_pcd])

In [3]:
# Random pick some point as bad grasp
points = np.asarray(cropped_pcd.points)

In [4]:
bad_conditions = []
for i in range(35):
    bad_condition = points[np.random.choice(len(points), 2, replace=False)]
    bad_conditions.append(bad_condition)

In [5]:
def sample_neighbor(kd_tree, points, seed, radius=0.03):
    new_points = np.zeros_like(seed)
    for i, point in enumerate(seed):
        indices = list(kd_tree.search_radius_vector_3d(point, radius)[1])
        choice = np.random.choice(indices,1)
        new_points[i] = points[choice]
    return new_points

seed = np.array([[0.15, 0, 0.05],[0.2, 0., 0]])
kd_tree = o3d.geometry.KDTreeFlann(cropped_pcd)
for i in range(5):
    bad_condition = sample_neighbor(kd_tree, points, seed)
    bad_conditions.append(bad_condition)
bad_conditions.append(seed)

for seed in bad_conditions[-6:]:
    for i in range(5):
        bad_condition = sample_neighbor(kd_tree, points, seed)
        bad_conditions.append(bad_condition)
bad_conditions = np.asarray(bad_conditions)
bad_conditions = bad_conditions.reshape(71,-1)

In [6]:
score_data = np.load("../score_function_data/new_score_data.npz")

In [7]:
conditions = score_data["conditions"]
scores = score_data["scores"]
pointclouds = score_data["point_clouds"]
idx = np.random.choice(len(points), 1024, replace=False)
down_points = points[idx].reshape(1,1024,3)
conditions = np.vstack([conditions, bad_conditions])
scores = np.hstack([scores, np.zeros(71)])
pointclouds = np.vstack([pointclouds]+[down_points]*71)

In [8]:
print(conditions.shape)
print(scores.shape)
print(pointclouds.shape)

(434, 6)
(434,)
(434, 1024, 3)


In [10]:
scores[:163] += 0.2
scores.clip(0, 1)

array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
       0.2, 0.2, 0.7, 0.5, 0.2, 0.7, 0.7, 0.8, 0.2, 0.2, 0.5, 0.2, 0.2,
       0.6, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2,
       0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.5, 0.4, 0.7,
       0.8, 0.2, 0.2, 0.2, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
       0.2, 0.2, 0.2, 0.6, 0.2, 0.2, 0.2, 0.2, 0.2, 0.6, 0.4, 0.2, 0.7,
       0.2, 0.2, 0.6, 0.7, 0.7, 0.2, 0.7, 0.7, 0.6, 0.5, 0.7, 0.3, 0.5,
       0.4, 0.6, 0.3, 0.2, 0.7, 0.7, 0.3, 0.2, 0.2, 0.5, 0.2, 0.7, 0.7,
       0.2, 0.2, 0.2, 0.6, 0.3, 0.6, 0.2, 0.2, 0.3, 0.7, 0.6, 0.2, 0.9,
       0.2, 0.2, 0.6, 0.2, 0.2, 0.2, 0.5, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
       0.2, 0.5, 0.4, 0.2, 0.2, 0.2, 0.3, 0.4, 0.3, 0.7, 1. , 0.7, 0.7,
       0.2, 1. , 0.2, 0.7, 0.8, 0.6, 0.2, 0.3, 0.2, 0.2, 0.2, 0.5, 0.2,
       0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.

In [11]:
np.savez("../score_function_data/score_data_augmented.npz", scores=scores, point_clouds=pointclouds, conditions=conditions)